In [ ]:
import backtrader as bt
import backtrader.feeds as btfeeds

import datetime
import pandas as pd

import schwabdev

import datetime
from dotenv import load_dotenv
import os

In [ ]:
class crossover(bt.Strategy):
    params = (
        ('printlog', True),
        ('ema_period', 20),
        ('sma_period', 50)
    )

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print(f"{dt.isoformat()} | {txt}") 

    def __init__(self):
        self.dataclose = self.data.close
        self.order = None
        self.comm = None;
        self.buy_price = None

        self.sma = bt.indicators.MovingAverageSimple(self.close, period=self.params.sma_period)
        self.ema = bt.indicators.ExponentialMovingAverage(self.close, period=self.params.ema_period)
        
        self.crossover = bt.indicators.CrossOver(self.ema, self.sma)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f"BUY EXECUTED | Price: {order.executed.price:.2f} | Cost: {order.executed.value:.2f} | Comm: {order.executed.comm:.2f}")
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f"SELL EXECUTED | Price: {order.executed.price:.2f} | Cost: {order.executed.value:.2f} | Comm: {order.executed.comm:.2f}")
        elif order.status in [order.Cancelled, order.Margin, order.Rejected]:
            self.log("ORDER CANCELED / MARGIN / REJECTED")
        
        self.order = None

    def next(self):
        if self.order:
            return
        
        if not self.position:
            if self.crossover > 0:
                self.order = self.buy()

                self.log(f"BUY CREATED | Price: {self.dataclose[0]:.2f}")
                self.notify_order(self.order)
        else:
            if self.crossover < 0:
                self.order = self.sell()

                self.log(f"SELL CREATED | Price: {self.dataclose[0]:.2f}")
                self.notify_order(self.order)

    def stop(self):
        self.log(f'Ending Value: {self.broker.getvalue():.2f}', doprint=True)

In [ ]:
cerebro = bt.Cerebro()

cerebro.addstrategy(crossover)
cerebro.addsizer(bt.sizers.SizerFix, stake=20)
cerebro.broker.setcommission(commission=0.01)
cerebro.broker = bt.brokers.BackBroker(slip_perc=0.005)